In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime

In [2]:


# Function to accept user inputs
def get_user_inputs():
    # Get start and end dates
    start_date = input("Enter the start date (YYYY-MM-DD): ")
    end_date = input("Enter the end date (YYYY-MM-DD): ")

    # Validate date format
    try:
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
        if start_date >= end_date:
            print("Start date must be earlier than the end date.")
            return None, None, None
    except ValueError:
        print("Invalid date format. Please enter in YYYY-MM-DD.")
        return None, None, None

    # Get investment amount
    try:
        investment_amount = float(input("Enter the total investment amount (₹): "))
        if investment_amount <= 0:
            print("Investment amount must be a positive value.")
            return None, None, None
    except ValueError:
        print("Invalid amount. Please enter a valid number.")
        return None, None, None

    return start_date, end_date, investment_amount

# Function to process the data
def process_stock_data(input_file, start_date, end_date, investment_amount, output_file):
    # Load the input CSV
    try:
        stock_data = pd.read_csv(input_file)
    except Exception as e:
        print(f"Error reading input file: {e}")
        return

    # Validate the CSV structure
    if "Ticker" not in stock_data.columns or "Weightage" not in stock_data.columns:
        print("Input file must have 'Ticker' and 'Weightage' columns.")
        return

    # Prepare for fetching data
    stock_data["Ticker"] = stock_data['Ticker'] + '.NS'
    tickers = stock_data["Ticker"].tolist()
    weightages = dict(zip(stock_data["Ticker"], stock_data["Weightage"]))

    # Fetch stock data using Yahoo Finance API
    print("Fetching stock data from Yahoo Finance...")
    try:
        stock_prices = yf.download(tickers, start=start_date, end=end_date, group_by="ticker", progress=False)
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return

    # Prepare a list for results
    results = []

    # Process each stock
    for ticker in tickers:
        if ticker in stock_prices.columns.levels[0]:
            print(f"Processing data for {ticker}...")
            ticker_data = stock_prices[ticker]["Close"]
            daily_shares = {}
            for date, close_price in ticker_data.items():
                if pd.isna(close_price) or close_price <= 0:  # Skip invalid prices
                    continue

                # Calculate investment amount and number of shares
                investment_in_stock = weightages[ticker] * investment_amount
                num_shares = investment_in_stock / close_price
                daily_shares[date.strftime("%Y-%m-%d")] = round(num_shares, 14)

            # Add the stock's data to results
            results.append({
                "Ticker": ticker.split(".")[0],  
                "Weightage": weightages[ticker],
                **daily_shares
            })
        else:
            print(f"No data available for {ticker}.")

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Save results to Excel
    results_df.to_excel(output_file, index=False)
    print(f"Output saved to {output_file}")

# Main program execution
if __name__ == "__main__":
    input_file = "Stocks.csv"  # Path to uploaded file
    output_file = "Investment_TimeTable_Stocks_Rows.xlsx"

    # Get user inputs
    start_date, end_date, investment_amount = get_user_inputs()

    if start_date and end_date and investment_amount:
        process_stock_data(input_file, start_date, end_date, investment_amount, output_file)


Fetching stock data from Yahoo Finance...
Processing data for ADANIPORTS.NS...
Processing data for APOLLOHOSP.NS...
Processing data for ASIANPAINT.NS...
Processing data for AXISBANK.NS...
Processing data for BAJAJFINSV.NS...
Processing data for BAJFINANCE.NS...
Processing data for BHARTIARTL.NS...
Processing data for BPCL.NS...
Processing data for BRITANNIA.NS...
Processing data for CIPLA.NS...
Processing data for COALINDIA.NS...
Processing data for DIVISLAB.NS...
Processing data for DRREDDY.NS...
Processing data for EICHERMOT.NS...
Processing data for GRASIM.NS...
Processing data for HCLTECH.NS...
Processing data for HDFCBANK.NS...
Processing data for HDFCLIFE.NS...
Processing data for HEROMOTOCO.NS...
Processing data for HINDALCO.NS...
Processing data for HINDUNILVR.NS...
Processing data for ICICIBANK.NS...
Processing data for INDUSINDBK.NS...
Processing data for INFY.NS...
Processing data for ITC.NS...
Processing data for JSWSTEEL.NS...
Processing data for KOTAKBANK.NS...
Processing